In [1]:
import os
import sys
import psi4
import numpy as np

sys.path.insert(0, "./common")
modpaths = os.environ.get('MODS_PATH')

if modpaths is not None :
    for path in modpaths.split(";"):
        sys.path.append(path)
from scf_run import run
from init_run import initialize

bset,bsetH, molelecule_str, psi4mol, wfn, jkobj = initialize(True,'DIRECT','cc-pvdz','3-21G','H2O1.xyz',\
                   'hf','hf',0,eri=None,rt_HF_iexch=False)

mints = psi4.core.MintsHelper(bset)

S = np.array(mints.ao_overlap())
numbas = bset.nbf()

I_Size = (numbas**4) * 8.e-9
nbfA = bsetH.nbf()

#make U matrix
U = np.eye(numbas)
S11=S[:nbfA,:nbfA]
S11_inv=np.linalg.inv(S11)
S12 =S[:nbfA,nbfA:]
P=np.matmul(S11_inv,S12)
U[:nbfA,nbfA:]=-1.0*P

#S block orthogonal
Stilde= np.matmul(U.T,np.matmul(S,U))


High Level functional : hf

Low Level functional : hf

Low Level basis : 3-21G

High Level basis : cc-pvdz

0 1
O1   0.000000000000   -0.000014000000   -0.348240000000
H1   0.000000000000   0.760011000000   -0.932852000000
H                     0.000000000000    -0.759996000000    -0.932908000000
symmetry c1
no_reorient
no_com

  Memory set to   1.863 GiB by Python driver.
    Molecular point group: c1
    Full point group: Cs

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
         O            0.000000000000    -0.000014000000    -0.348240000000    15.994914619570
         H            0.000000000000     0.760011000000    -0.932852000000     1.007825032230
         H            0.000000000000    -0.759996000000    -0.932908000000     1.007825032230

centers of high level subsys: 2
Number

In [2]:
mints = psi4.core.MintsHelper(bset)
#intialize fock_factory
from Fock_helper import fock_factory
Hcore = np.asarray(mints.ao_potential()) + np.asarray(mints.ao_kinetic())

In [3]:
from Fock_helper import fock_factory
Cocc = np.array(wfn.Ca_subset('AO','OCC'))
Dmat_test= np.matmul(Cocc,Cocc.T)
fock_base = fock_factory(jkobj,Hcore,S, \
                            funcname='hf',basisobj=bset,exmodel=0)

Fock_test = fock_base.get_bblock_Fock(None,np.array(wfn.Da()),func_acc='hf',basis_acc=bsetH,U=np.eye(U.shape[0]))
np.allclose(np.array(wfn.Da()),Dmat_test)

True

In [4]:
Fa=np.array(wfn.Fa())

In [5]:
np.allclose(Fa,Fock_test)

True

In [6]:
fock_base_BO = fock_factory(jkobj,Hcore,Stilde, \
                            funcname='hf',basisobj=bset,exmodel=0)

U_inv = np.linalg.inv(U)

Dmat_BO = np.matmul(U_inv,np.matmul(Dmat_test,U_inv.T))
Fock_test = fock_base_BO.get_bblock_Fock(None,Dmat_BO,func_acc='hf',basis_acc=bsetH,U=U)


In [7]:
np.allclose(np.matmul(U.T,np.matmul(Fa,U)),Fock_test)

True